In [ ]:
import pandas as pd

In [2]:
def data_preprocessing(data):

  data.columns=[col.lower() for col in data.columns]
  for row in data.index:
    for col in data.columns:
      if isinstance(data[col][row],str):
        data.loc[row, col]=data[col][row].lower()
        data.loc[row, col]=data[col][row].replace("\n"," ")
        data.loc[row, col]=data[col][row].replace("\r"," ")
        data.loc[row, col]=data[col][row].replace("\t"," ")

  return data

In [4]:
df= pd.read_csv("/data/training_data.csv")
df.head()

,name,description,source_table
0,Relax In Turkish Hammam Bath and Full-body Mas...,Indulge in a soothing spa experience during yo...,activities
1,Orange Bay Full Day with Snorkeling and lunch ...,Make the most of Hurghada's beautiful weather ...,activities
2,Hurghada: Sea and Mountains ATV Quad Bike Tour,Go on an exhilarating ride through the Red Sea...,activities
3,Hurghada a Turkish Hammam And Spa With Transfers,"Personalized Luxury Enjoy a private sauna, Ham...",activities
4,"Moroccan Bath Package Massage, Jacuzzi, Sauna,...",Unwind with a luxurious Moroccan bath! Enjoy a...,activities


In [7]:
processed_data=data_preprocessing(df)

In [9]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
# Import Dataset from the datasets library
from datasets import Dataset

def finetune_sbert(data):
    # Initialize base SBERT model
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # Prepare training examples - specifically for pandas DataFrame
    train_examples = []

    # Create examples from pandas DataFrame
    for _, row in data.iterrows():
        if 'name' in row and 'description' in row:
            # Create positive pairs (name and description should be semantically similar)
            train_examples.append(InputExample(
                texts=[str(row['name']), str(row['description'])],  # Convert to string to ensure compatibility
                label=1.0
            ))

    # Create data loader
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

    # Use MultipleNegativesRankingLoss which works better for semantic similarity
    train_loss = losses.MultipleNegativesRankingLoss(model)

    # Train the model
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=5,
        warmup_steps=100,
        optimizer_params={'lr': 2e-5}
    )

    # Save the fine-tuned model
    model.save('fine-tuned-sbert-for-tourism-data')

    return model

In [10]:
finetune_sbert(processed_data)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
500,0.985100
1000,0.607100


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)